In [1]:
# This is needed for deferring annotation parsing in TVMScript
from __future__ import annotations
import numpy as np
import tvm
from tvm import relax
from tvm.ir.module import IRModule
from tvm.script import relax as R
from tvm.script import tir as T

In [2]:
import timeit

In [17]:
from tvm import te
from tvm import topi

参考文章(TIR)：
- https://mlc.ai/zh/chapter_tensor_program/case_study.html
- https://mlc.ai/zh/chapter_end_to_end/index.html

参考文章(TE)：
- https://tvm.hyper.ai/docs/how_to/optimize/cpu_conv
- https://tvm.hyper.ai/docs/how_to/te_schedules/primitive

In [3]:
@tvm.script.ir_module
class MyModule:
    @T.prim_func
    def main(
        A: T.Buffer[(1024, 1024), "float32"],
        B: T.Buffer[(1024, 1024), "float32"],
        C: T.Buffer[(1024, 1024), "float32"],
    ):
        T.func_attr({"global_symbol": "main", "tir.noalias": True})
        for i, j, k in T.grid(1024, 1024, 1024):
            with T.block("C"):
                vi, vj, vk = T.axis.remap("SSR", [i, j, k])
                with T.init():
                    C[vi, vj] = 0.0
                C[vi, vj] = C[vi, vj] + A[vi, vk] * B[vk, vj]

In [4]:
dtype = "float32"
a_np = np.random.rand(1024, 1024).astype(dtype)
b_np = np.random.rand(1024, 1024).astype(dtype)

a_nd = tvm.nd.array(a_np)
b_nd = tvm.nd.array(b_np)
c_nd = tvm.nd.empty((1024, 1024), dtype="float32")

numpy执行矩阵乘法的花费时间：

In [5]:
np_repeat = 20
np_runing_time = timeit.timeit(
    setup="import numpy\n"
    "M = " + str(1024) + "\n"
    "K = " + str(1024) + "\n"
    "N = " + str(1024) + "\n"
    'dtype = "float32"\n'
    "a = numpy.random.rand(M, K).astype(dtype)\n"
    "b = numpy.random.rand(K, N).astype(dtype)\n",
    stmt="answer = numpy.dot(a, b)",
    number=np_repeat,
)
print("Numpy running time: %f" % (np_runing_time / np_repeat))

Numpy running time: 0.020675


tir基线，未经过优化的性能：

In [6]:
lib = tvm.build(MyModule, target="llvm")
f_timer_before = lib.time_evaluator("main", tvm.cpu())
print("Time cost of MyModule: %f s" % (f_timer_before(a_nd, b_nd, c_nd).mean))

Time cost of MyModule: 2.756632 s


开始优化：

一：分块
1. 对循环进行拆分：（按照L1 cache的大小，拆分出block）
2. 对循环进行重排序，(block spatial axis放在最内部, reduce axis放在最外部)

In [11]:
!lscpu | grep "cache" # 或者"缓存", 我这儿是32KB, 选择分配给Block 4 * 64 * 64 = 16KB 给Block

L1d 缓存：       32K
L1i 缓存：       32K
L2 缓存：        256K
L3 缓存：        12288K


In [12]:
def schedule_mm(sch: tvm.tir.Schedule, jfactor=4):
    block_C = sch.get_block("C", "main")
    i, j, k = sch.get_loops(block=block_C)
    j_0, j_1 = sch.split(loop=j, factors=[None, jfactor])
    sch.reorder(i, j_0, k, j_1)
    sch.decompose_reduction(block_C, k)
    return sch

def schedule_mm2(sch: tvm.tir.Schedule, bn=64, kfactor=4):
    block_C = sch.get_block("C", "main")
    i, j, k = sch.get_loops(block=block_C)
    # 对spatial进行循环拆分, bn*bn*sizeof(float)=4KB, L1 cache 
    i_0, i_1 = sch.split(loop=i, factors=[None, bn])
    j_0, j_1 = sch.split(loop=j, factors=[None, bn])
    # 对reduce进行循环拆分
    k_0, k_1 = sch.split(loop=k, factors=[None, kfactor])
    # 进行循环重排序, 将 reduction 域提升到 分块循环 之外
    sch.reorder(i_0, j_0, k_0, k_1, i_1, j_1)
    return sch

In [16]:
sch = tvm.tir.Schedule(MyModule)
sch = schedule_mm2(sch, bn=64)

In [17]:
sch.mod.show()

In [18]:
lib = tvm.build(sch.mod, target="llvm")
f_timer_after = lib.time_evaluator("main", tvm.cpu())
print("Time cost of MyModule=>schedule_mm: %f s" % (f_timer_after(a_nd, b_nd, c_nd).mean))

Time cost of MyModule=>schedule_mm: 0.336546 s


二、向量化

另一个重要技巧是向量化，当内存访问模式一致时，编译器可以检测到这种模式并将连续内存传递给向量处理器。TVM 中可以用 `vectorize` 接口来提示编译器这种模式，这样就可以进行加速。

本教程选择向量化内部循环 row data（对缓存更友好）。

In [22]:
def schedule_mm3(sch: tvm.tir.Schedule, bn=64, kfactor=4):
    block_C = sch.get_block("C", "main")
    i, j, k = sch.get_loops(block=block_C)
    # 一、分块
    # 对spatial进行循环拆分, bn*bn*sizeof(float)=4KB, L1 cache 
    i_0, i_1 = sch.split(loop=i, factors=[None, bn])
    j_0, j_1 = sch.split(loop=j, factors=[None, bn])
    # 对reduce进行循环拆分
    k_0, k_1 = sch.split(loop=k, factors=[None, kfactor])
    # 进行循环重排序, 将 reduction 域提升到 分块循环 之外
    sch.reorder(i_0, j_0, k_0, k_1, i_1, j_1)

    # 二、向量化：向量化内部循环(对缓存更友好)
    sch.vectorize(j_1)
    return sch

In [23]:
sch = tvm.tir.Schedule(MyModule)
sch = schedule_mm3(sch, bn=64)

In [24]:
lib = tvm.build(sch.mod, target="llvm")
f_timer_after = lib.time_evaluator("main", tvm.cpu())
print("Time cost of MyModule=>schedule_mm: %f s" % (f_timer_after(a_nd, b_nd, c_nd).mean))

Time cost of MyModule=>schedule_mm: 0.322161 s


In [26]:
sch.mod.show()

三、循环置换

查看上面的 IR，可以看到内部循环的 row data 对于 B 和 C 都是向量化的（影响到了vi, vj, 进而对B的读和C的写实现了向量化）。

接下来查看 A 的访问模式。在当前调度中，A 是逐列访问的，但它对缓存不友好。（因为抛去向量化的j_1后，i_1是最内部的循环，i_1的更新导致了vi的更新，进而导致了A的逐列访问）

**如果改变 k_1 和内轴 i_1 的嵌套循环顺序，A 矩阵的访问模式对缓存更友好**。

In [138]:
def schedule_mm4(sch: tvm.tir.Schedule, bn=64, kfactor=4):
    block_C = sch.get_block("C", "main")
    i, j, k = sch.get_loops(block=block_C)
    # 一、分块
    # 对spatial进行循环拆分, bn*bn*sizeof(float)=4KB, L1 cache 
    i_0, i_1 = sch.split(loop=i, factors=[None, bn])
    j_0, j_1 = sch.split(loop=j, factors=[None, bn])
    # 对reduce进行循环拆分
    k_0, k_1 = sch.split(loop=k, factors=[None, kfactor])
    # 进行循环重排序, 将 reduction 域提升到 分块循环 之外
    sch.reorder(i_0, j_0, k_0, k_1, i_1, j_1)

    # 二、向量化：向量化内部循环(对缓存更友好)
    sch.vectorize(j_1)

    # 三、循环置换，改变A矩阵的访问模式, 将逐列访问访问逐行访问
    sch.reorder(i_0, j_0, k_0, i_1, k_1, j_1)
    return sch

In [139]:
sch = tvm.tir.Schedule(MyModule)
sch = schedule_mm4(sch, bn=64)

In [140]:
lib = tvm.build(sch.mod, target="llvm")
f_timer_after = lib.time_evaluator("main", tvm.cpu())
print("Time cost of MyModule=>schedule_mm: %f s" % (f_timer_after(a_nd, b_nd, c_nd).mean))

Time cost of MyModule=>schedule_mm: 0.212235 s


In [9]:
sch.mod.show()

四、数组打包

> 解决B的访问模式

另一个重要的技巧是数组打包，对多维数组的存储进行重新排序，展平并存储在一维内存中，方便顺序访问。

当前B的访问模式：在做BLOCK内部的矩阵乘法时（[64, 4\*64]@[64\*4, 64]）：
- 此时A的第二个维度4*64在一行中；
- 而此时B的第一个维度的64*4不在一个维度中，而在一个[4, 64]的block中；

![](../pics/array-packing.png)

观察展平后 B 的数组访问模式，当迭代 K 维时，它不是顺序的。

可以用维度 [K][N] 对 B 重新排序，使其具有 [N/bn][K][bn] 维度，其中 bn 是分块因子，也是内循环中 B 的向量大小。

这种重新排序将 N 拆分为两个维度——bigN（N/bn）和 littleN（bn）——新维度 [N/bn][K][bn] 匹配 B 从外部到内部循环的索引（no, ko, ki, ni) 在展平后导致 B 的顺序访问模式。

【注意】：
- 这里实现的关键在于对[N/bn]的并行访问，才提高了效率；?????感觉这里做的不如直接转置效果好

In [194]:
def te_matmul(A: te.Tensor, B: te.Tensor, bn=64) -> te.Tensor:
    assert A.shape[1] == B.shape[0]
    m = A.shape[0]
    n = B.shape[1]
    K = A.shape[1]
    packedB = te.compute((tvm.tir.indexdiv(n, bn), K, bn), lambda bigN, k, littleN: B[k, bigN * bn + littleN], name="packedB")
    k = te.reduce_axis((0, K), name="k")
    # 提前将j的block轴拆分出来
    return te.compute((m, tvm.tir.indexdiv(n, bn), bn), 
                lambda i, j_0, j_1: te.sum(
                    A[i, k] * packedB[j_0, k, j_1], axis=k), name="C")

# 注意，此时需要重新编写mm的算法
def schedule_mm5(sch: tvm.tir.Schedule, bn=64, kfactor=4):
    block_C = sch.get_block("C", "main")
    i, j_0, j_1, k = sch.get_loops(block=block_C)
    # 一、分块
    # 对spatial进行循环拆分, bn*bn*sizeof(float)=4KB, L1 cache 
    i_0, i_1 = sch.split(loop=i, factors=[None, bn])
    # j_0, j_1 = sch.split(loop=j, factors=[None, bn]), 因为前面matmul已经拆分过了, 所以不再拆分j
    # 对reduce进行循环拆分
    k_0, k_1 = sch.split(loop=k, factors=[None, kfactor])
    # 进行循环重排序， 将 reduction 域提升到 分块循环 之外
    sch.reorder(i_0, j_0, k_0, k_1, i_1, j_1)

    # 二、向量化：向量化内部循环(对缓存更友好)
    sch.vectorize(j_1)

    # 三、循环置换：改变A矩阵的访问模式, 将逐列访问访问逐行访问
    sch.reorder(i_0, j_0, k_0, i_1, k_1, j_1)

    # 四、数组打包：改变B矩阵的访问模式，下面的sch只是对packB做优化, 
    block_B = sch.get_block("packedB", "main")
    b_i, _, b_k = sch.get_loops(block=block_B)
    sch.parallel(b_i)
    sch.vectorize(b_k)

    return sch

In [195]:
A = te.placeholder((1024, 1024), name="A", dtype="float32")
B = te.placeholder((1024, 1024), name="B", dtype="float32")
C = te_matmul(A, B, bn=64)
te_matmul_func = te.create_prim_func([A, B, C])
MyModule2 = tvm.IRModule({"main": te_matmul_func})

In [196]:
sch2 = tvm.tir.Schedule(MyModule2)
sch2 = schedule_mm5(sch2, bn=64)

In [197]:
# 这里的C
c_nd = tvm.nd.empty((1024, 1024/64, 64), dtype="float32")

lib = tvm.build(sch2.mod, target="llvm")
f_timer_after = lib.time_evaluator("main", tvm.cpu())
print("Time cost of MyModule=>schedule_mm: %f s" % (f_timer_after(a_nd, b_nd, 
        c_nd).mean))

Time cost of MyModule=>schedule_mm: 0.168757 s


In [198]:
sch2.mod.show()

五、块的写缓存

分块后，程序会逐块将结果写入 C（访问模式不是顺序的）

因此，可以使用顺序缓存数组来保存块结果，并在所有块结果准备好时写入 C。

In [251]:
def schedule_mm6(sch: tvm.tir.Schedule, bn=64, kfactor=4):
    block_C = sch.get_block("C", "main")
    # 六、分配写缓存块
    C_global = sch.cache_write(block_C, 0, "global")

    i, j_0, j_1, k = sch.get_loops(block=block_C)
    
    # 一、分块
    # 对spatial进行循环拆分, bn*bn*sizeof(float)=4KB, L1 cache 
    i_0, i_1 = sch.split(loop=i, factors=[None, bn])
    k_0, k_1 = sch.split(loop=k, factors=[None, kfactor])
    # 进行循环重排序， 将 reduction 域提升到 分块循环 之外
    sch.reorder(i_0, j_0, k_0, k_1, i_1, j_1)
    
    # 六、循环展开：增加并发执行的机会
    sch.unroll(k_1)

    # 二、向量化：向量化内部循环(对缓存更友好)
    sch.vectorize(j_1)

    # 三、循环置换：改变A矩阵的访问模式, 将逐列访问访问逐行访问
    sch.reorder(i_0, j_0, k_0, i_1, k_1, j_1)

    # 四、数组打包：改变B矩阵的访问模式，下面的sch只是对packB做优化, 
    block_B = sch.get_block("packedB", "main")
    b_i, _, b_k = sch.get_loops(block=block_B)
    sch.parallel(b_i)
    sch.vectorize(b_k)
    
    # 六、写缓存块：改变C矩阵的写入 
    sch.reverse_compute_at(C_global, j_0)
    _, _, c_0, c_1 = sch.get_loops(block=C_global)
    sch.vectorize(c_1)

    return sch

In [252]:
sch2 = tvm.tir.Schedule(MyModule2)
sch2 = schedule_mm6(sch2, bn=64)

In [253]:
c_nd = tvm.nd.empty((1024, 1024/64, 64), dtype="float32")

lib = tvm.build(sch2.mod, target="llvm")
f_timer_after = lib.time_evaluator("main", tvm.cpu())
print("Time cost of MyModule=>schedule_mm: %f s" % (f_timer_after(a_nd, b_nd, 
        c_nd).mean))

Time cost of MyModule=>schedule_mm: 0.139549 s


六、并行化

此外，还可以利用多核处理器进行线程级并行化。

In [255]:
def schedule_mm7(sch: tvm.tir.Schedule, bn=64, kfactor=4):
    block_C = sch.get_block("C", "main")
    # 六、分配写缓存块
    C_global = sch.cache_write(block_C, 0, "global")

    i, j_0, j_1, k = sch.get_loops(block=block_C)
    
    # 一、分块
    # 对spatial进行循环拆分, bn*bn*sizeof(float)=4KB, L1 cache 
    i_0, i_1 = sch.split(loop=i, factors=[None, bn])
    k_0, k_1 = sch.split(loop=k, factors=[None, kfactor])
    # 进行循环重排序， 将 reduction 域提升到 分块循环 之外
    sch.reorder(i_0, j_0, k_0, k_1, i_1, j_1)
    
    # 六、循环展开：增加并发执行的机会
    sch.unroll(k_1)

    # 二、向量化：向量化内部循环(对缓存更友好)
    sch.vectorize(j_1)

    # 三、循环置换：改变A矩阵的访问模式, 将逐列访问访问逐行访问
    sch.reorder(i_0, j_0, k_0, i_1, k_1, j_1)

    # 四、数组打包：改变B矩阵的访问模式，下面的sch只是对packB做优化, 
    block_B = sch.get_block("packedB", "main")
    b_i, _, b_k = sch.get_loops(block=block_B)
    sch.parallel(b_i)
    sch.vectorize(b_k)
    
    # 六、写缓存块：改变C矩阵的写入 
    sch.reverse_compute_at(C_global, j_0)
    _, _, c_0, c_1 = sch.get_loops(block=C_global)
    sch.vectorize(c_1)

    # 七、写缓存块：加入并行
    sch.parallel(c_0)

    return sch

In [256]:
sch2 = tvm.tir.Schedule(MyModule2)
sch2 = schedule_mm7(sch2, bn=64)

In [257]:
c_nd = tvm.nd.empty((1024, 1024/64, 64), dtype="float32")

lib = tvm.build(sch2.mod, target="llvm")
f_timer_after = lib.time_evaluator("main", tvm.cpu())
print("Time cost of MyModule=>schedule_mm: %f s" % (f_timer_after(a_nd, b_nd, 
        c_nd).mean))

Time cost of MyModule=>schedule_mm: 0.150843 s
